In [ ]:
modello_per_analisi = ''
# Opzioni possibili: Mercedes_Classe_A, BMW_Serie_1, Audi_A3, Opel_Corsa

### 0. Import funzioni e librerie

In [ ]:
import pandas as pd
import os
import sys
import requests
import importlib
from pathlib import Path

In [ ]:
source_root = Path.cwd().parents[0]
# Percorsi delle cartelle contenenti i moduli
loader_dir = source_root / '2_Data_Preparation' / 'Utils'
utils_dir = source_root / '3_Data_Analysis' / 'Utils'

# Aggiungi al sys.path
sys.path.extend([str(loader_dir), str(utils_dir)])

# Import
from data_loader import load_csv_for_notebook
from Plots_maker import plot_is_conveniente_distribution, plot_price_prediction_distributions
from Function_price_analysis import modello_ml, predizione_prezzo, is_conveniente_class_report, indice_appetibilita, display_top_auto

In [ ]:
# --- IMPORT CONFIGURAZIONE DELL'AUTO ---
# Percorsi delle cartelle contenenti i moduli
project_root = Path.cwd().parents[1]
config_path = project_root / 'Data'/ 'config'
# Aggiungi al sys.path
sys.path.extend([str(config_path)])
print('Il config_dir risulta:', config_path, ',')
print("il cui contenuto della cartella:", os.listdir(config_path))
config_path
# --- Import dinamico del modulo di configurazione in base al modello ---
modulo_config = f"config_{modello_per_analisi}"  # esempio: config_Mercedes_Classe_A
try:
    config = importlib.import_module(modulo_config)
    print(f"✅ Modulo di configurazione '{modulo_config}' caricato.")
except ModuleNotFoundError:
    print(f"❌ Modulo di configurazione '{modulo_config}' non trovato.")
    sys.exit(1)
# --- Accesso ai parametri di configurazione ---
#allestimenti = config.allestimenti
allestimento_performance = config.allestimento_performance
allestimento_sport = config.allestimento_sport
allestimento_middle = config.allestimento_middle
allestimento_base = config.allestimento_base
motorizzazioni_norm = config.motorizzazioni_norm
modelli_ord = config.modelli_ord
mappa_cv = config.mappa_cv
mappa_allestimenti = config.mappa_allestimenti

### 1. Dati

In [ ]:
#### CARICAMENTO DATI SINGOLI
data_dummy = pd.read_csv(project_root / 'Data' / 'Processed_data' / modello_per_analisi / f'data_dummy_{modello_per_analisi}.csv')
data = pd.read_csv(project_root / 'Data' / 'Processed_data' / modello_per_analisi / f'data_{modello_per_analisi}.csv')

X = data_dummy.drop(columns=['Prezzo', 'Distanza', 'Venditore'])
y = data_dummy['Prezzo']

### 2. Predizione prezzo, analisi della convenienza

In [ ]:
modello_ml(X, y, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))
data_ml, y_pred = predizione_prezzo(data_dummy, X, (project_root / 'Data' / 'Modelli_ML' / modello_per_analisi))

In [ ]:
plot_is_conveniente_distribution(data_ml)

In [ ]:
plot_price_prediction_distributions(data_ml)

In [ ]:
#is_conveniente_class_report(data_ml)

### 3. Indice di attendibilità

In [ ]:
#Prezzo soglia: prezzo oltre il quale il punteggio di attendibilità diminuisce
prezzo_soglia=35000

#Valori per il calcolo indice di attendibilità
valore_idx_anni = 0.025
valore_idx_prezzo = 0.15
valore_idx_km= 0.15 
valore_idx_dist = 0.10 
valore_idx_allestimento= 0.25 
valore_idx_cv =0.025
valore_idx_cambio = 0.20 

In [ ]:
df = indice_appetibilita(data, y_pred, prezzo_soglia, valore_idx_anni, valore_idx_prezzo, valore_idx_km, 
                           valore_idx_dist, valore_idx_allestimento, valore_idx_cv, valore_idx_cambio, 
                           allestimento_performance, allestimento_sport,allestimento_middle,allestimento_base)

In [ ]:
prezzo_max = 12000
prezzo_min = 7000
km_max =90000
km_min = 0
dist_max = 150
dispaly = 20

display_top_auto(df, dispaly, prezzo_max, prezzo_min, km_max, km_min, dist_max, carburante=None)
